In [ ]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pylab as plt
import sys, os, glob
import numpy as np
import subprocess

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

from IPython.display import HTML
import xml.etree.ElementTree as ET
try:
    tree = ET.parse(os.environ['HADOOP_CONF_DIR'] + '/yarn-site.xml')
except IOError:
    raise IOError("Can't find the yarn configuration -- is HADOOP_CONF_DIR set?")
root = tree.getroot()
yarn_web_app = root.findall("./property[name='yarn.resourcemanager.webapp.address']")[0].find('value').text
yarn_web_app_string = "If this works successfully, you can check the <a target='_blank' href='http://{yarn_web_app}'>YARN application scheduler</a> and you should see your app listed there. Clicking on the 'Application Master' link will bring up the familiar Spark Web UI. "

plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

# Analyzing the Gutenberg Books Corpus

In this notebook, we will use the cleaned, pre-processed data that we created in the [pre-processing part](gutenberg-preprocessing-SOLUTIONS.ipynb). As a reminder, we ended up with an RDD of `(gid, text)` tuples that has been cleaned and we stored it on HDFS at `/user/<YOUR_USERNAME>/gutenberg/cleaned_rdd`. 

In the [first analysis notebook](gutenberg-analysis-SOLUTIONS.ipynb) we build an N-gram viewer for the gutenberg books project. Now, we will use the corpus to train a simple language classification model using [Spark's machine learning library](http://spark.apache.org/docs/latest/mllib-guide.html).

## Set up and launch the Spark runtime

Remember from the previous notebook that we have a saved configuration in `./spark_config/` -- so all we need to do is set the `SPARK_CONF_DIR` environment variable and our default configuration will be used: 

In [ ]:
# specify the configuration directory
os.environ['SPARK_CONF_DIR'] = os.path.realpath('./spark_config')

# how many cores do we have for the driver
ncores = int(os.environ.get('LSB_DJOB_NUMPROC', 1)) 

# here we set the memory we want spark to use for the driver JVM
os.environ['SPARK_DRIVER_MEMORY'] = '%dG'%(ncores*2*0.7)

# we have to tell spark which python executable we are using
os.environ['PYSPARK_PYTHON'] = subprocess.check_output('which python', shell=True).rstrip()

import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext

In [ ]:
conf = SparkConf()

sc = SparkContext(master='yarn-client', conf=conf)

HTML(yarn_web_app_string.format(yarn_web_app=yarn_web_app))

## Load the data from HDFS

In [ ]:
# TODO: load cleaned_rdd from the HDFS
cleaned_rdd = <FILL IN>

### A brief aside - correcting for data skew

Before we begin the loop-heavy analysis, we can do one more bit of optimization. Not all of the books in the corpus are the same length. This means that when we extract the ngrams, the differences will become even more pronounced. Since our basic unit of work here is a full book, this will result in uneven task execution times. Furthermore, because an individual "task" will actually process many books, this means that shorter books will be stuck in the queue behind longer books. 

To visualize this, lets count the number of characters in each partition: 

In [ ]:
def count_chars_in_partition(iterator): 
    """
    Sum up the string lengths in each partition
    """
    c = 0
    for (gid, text) in iterator: 
        c+=len(text)
    yield c    

In [ ]:
plt.hist(cleaned_rdd.mapPartitions(count_chars_in_partition).collect())
plt.xlabel('Number of characters per partition')
plt.ylabel('Number of partitions')

It's easy to see that some of the partitions have much more data than others. You can also verify this by looking at the Spark web UI for your job and clicking on the last stage that just executed (it will be named "collect"). You should see something like this at the top of the page: 

![Task summary](../../slides/figs/summary_tasks.png)

You can see that the longest tasks are taking several times more to execute than the mean even for this extremely simple operation. The data skew is clear if you scroll down a bit more and see the actual data input into each task and sort by duration: 

![Task duration](../../slides/figs/task_duration.png)

The longest few tasks are processing considerably more data than even the next several. Such a data skew may not matter for certain applications, but in our case we can expect it to make a difference. 

To try and correct it, we must first realize that for our application we have very few requirements about text order. We don't care about the order of words, as long as the documents are tagged with the correct language. So splitting a book into two and processing them out of order doesn't really matter. 

With this in mind, we can use a function like `chunk_text` below that creates approximately equal-sized "blocks" of text. The blocking means we'll get some spurious character ngrams at the beginning and end of each document, but if we block the data in large enough chunks this shouldn't matter too much.

In [ ]:
def chunk_text(text, block_len=10000) : 
    """
    Ensure that blocks of text are no larger than `block_len`.
    
    Arguments: 
        
        text: the raw text string
    """
    len_text = len(text)
    
    if len_text < block_len : 
        return text
    else : 
        nblocks = len_text/block_len
        return [text[i*block_len:(i+1)*block_len] for i in xrange(nblocks)]

Now we apply this function to the `cleaned_rdd` and visualize the partition size distribution again. Since we want to keep the keys untouched, we can use the `flatMapValues` RDD method. However, since this method keeps the same data within each partition, we also need to *repartition* the data afterwards to randomize the chunk distributions. 

In [ ]:
plt.hist(cleaned_rdd.flatMapValues(chunk_text)
                    .repartition(400)
                    .mapPartitions(count_chars_in_partition)
                    .collect())
plt.xlabel('Number of characters per partition')
plt.ylabel('Number of partitions')

The distribution is now still not perfect, but we have corrected it considerably. Lets make a restructured version of the full dataset and cache it: 

In [ ]:
equalized_rdd = (cleaned_rdd.flatMapValues(chunk_text)
                            .setName('equalized_rdd')
                            .repartition(400)).cache()

### Load in the metadata dictionary and broadcast it

Just as in the previous notebook, we will load our pre-generated metadata dictionary and broadcast it to all the executors. 

In [ ]:
from cPickle import load

with open('{home}/gutenberg_metadata.dump'.format(home=os.environ['HOME']), 'r') as f :
    meta_dict = load(f)

In [ ]:
# TODO: create meta_b by broadcasting meta_dict
meta_b = <FILL IN>

Now, our `cleaned_rdd` contains `gid`'s as keys and text as values and if we want some other piece of metadata, we can just access it via the lookup table, for example `meta_b.value[gid][meta_name]`. 

We will use the same `extract_ngrams` and `vectorize_doc` functions as in the previous notebook: 

In [ ]:
from pyspark.mllib.linalg import SparseVector

def extract_ngrams(tokens, ngram_range=[1,1], select_ngrams = None, ngram_type='word'):
    """
    Turn tokens into a sequence of n-grams 

    Arguments:

        tokens: a list of tokens

    Optional Keywords:

        ngram_range: a tuple with min, max ngram ngram_range
    
        select_ngrams: the vocabulary to use
    
        ngram_type: whether to produce word or character ngrams

    Output:

    Generator yielding a list of ngrams in the desired range
    generated from the input list of tokens

    """
    join_str = "" if ngram_type=='character' else " "
    
    # handle token n-grams
    min_n, max_n = ngram_range
    n_tokens = len(tokens)
    
    for n in xrange(min_n, min(max_n + 1, n_tokens + 1)):
        for i in xrange(n_tokens - n + 1):
            if n == 1: 
                res = tokens[i]
            else : 
                res = join_str.join(tokens[i: i+n])
           
            # if we are using a lookup vocabulary, check for membership here
            if select_ngrams is not None : 
                if res in select_ngrams: 
                    yield res
            else : 
                yield res
            
def vectorize_doc(doc, vocab, ngram_range = [1,1], ngram_type='word') : 
    """
    Returns a vector representation of `doc` given the reference 
    vocabulary `vocab` after tokenizing it with `tokenizer`
    
    Arguments: 
        
        doc: a sequence of tokens (words or characters)
        
        vocab: the vocabulary mapping
        
    Keywords:
    
        ngram_range: the range of ngrams to process
        
        ngram_type: whether to produce character or word ngrams; default is 
        
    Returns:
    
        a sparse vector representation of the document given the vocabulary mapping
    """
    from collections import defaultdict
    from scipy.sparse import csr_matrix 
        
    # count all the word occurences 
    data = np.zeros(len(vocab))
    
    for ngram in extract_ngrams(doc, ngram_range, vocab, ngram_type) : 
         data[vocab[ngram]] += 1
            
    return data

# Language classification

Here we will try to use some of the same techniques we developed before, but apply them to a classification problem: determining whether a text is English or German. 

We will use the rather straightforward method outlined in [Cavnar & Trenkle 1994](http://odur.let.rug.nl/~vannoord/TextCat/textcat.pdf):

For each of the English/German training sets:

1. tokenize the text (spaces are also tokens, so we replace them with "_")
2. extract N-grams where 1 < N < 5
3. determine the most common N-grams for each corpus
4. encode both sets of documents using the combined top ngrams



In the last notebook, we used words as "tokens" -- now we will use characters, even accounting for white space (which we will replace with "_"). We will use the two example sentences again:

    document 1: "a dog bit me"
    document 2: "i bit the dog back"
    
First, we use the `extract_ngrams` function: 

In [ ]:
s1 = "a dog bit me"
s2 = "i bit the dog back"

In [ ]:
ngrams1 = list(extract_ngrams(s1.replace(' ','_'), ngram_range=[1,5], ngram_type='character'))
ngrams2 = list(extract_ngrams(s2.replace(' ','_'), ngram_range=[1,5], ngram_type='character'))

In [ ]:
print(list(ngrams1))

We can create the vocabulary by getting the set of all ngrams and building a lookup table:

In [ ]:
vocab = set(ngrams1) | set(ngrams2)

In [ ]:
vocab_dict = {word:ind for ind,word in enumerate(vocab)}
print('number of ngrams: ',len(vocab_dict))

Note that extracting ngrams can increase the size of the data quite a lot!

In [ ]:
vocab_dict

And finally, we can use `vectorize_doc` with the vocabulary mapping to turn the documents into vectors: 

In [ ]:
vectorize_doc(s1.replace(' ','_'), vocab_dict, ngram_range=[1,5], ngram_type='character')

### Saving memory consumption with `mapPartitions`

As you could see from the simple example of extracting ngrams from two short phrases above, by extracting the 1-5 grams from a simple 12-character string, we created a vector with 48 stored values. Our actual documents will therefore swell in size very rapidly -- we definitely don't want to be holding all of those huge lists in memory! 

Remember, our first goal is to get the top most-used N-grams. For this, we just need to create an RDD of N-grams and to avoid building lists we'll use the technique of generators discussed on the first day. 

Note that the `extract_ngrams` function above is already a generator: now we just want to make a small wrapper function that uses `extract_ngrams` to "yield" ngrams one by one into the RDD. 

A slight complication is that `mapPartitions` gives us a partition *iterator* over the data in the partition. This iterator will give us partition data one element at a time, which in this case are just the individual documents. We can pass each of these documents to `extract_ngrams`. 

In [ ]:
from collections import defaultdict

def ngram_generator(iterator, ngram_range=[1,1], ngram_type='word') : 
    """Take an iterator of documents and create a generator of ngrams
    
    Arguments:
        
        iterator: the document iterator
        
    Keywords:
        
        ngram_range: the range of ngrams to consider
        
        ngram_type: whether to extract word or character ngrams; default is word
    """ 
    for text in iterator : 
        for ngram in extract_ngrams(text.replace(' ', '_'), ngram_range, ngram_type=ngram_type): 
            yield ngram

We will subsample the english part of `equalized_rdd` in order to make this next set of cells complete in a reasonable amount of time -- once it's working, you can go back and do it for the full dataset, but it will take approximately 30 minutes. 

In [ ]:
# TODO: create english_rdd and german_rdd by filtering the equalized_rdd for 'en' and 'de' 
# make sure you only take a 10% sample of the english RDD by using the 'sample' method

english_rdd = equalized_rdd.<FILL IN>.setName('english_rdd').cache()
german_rdd  = equalized_rdd.<FILL IN>.setName('german_rdd').cache()

#### Making the sets of most common english and german ngrams

To build the sets of ngrams, we use the now-familiar pattern: 

1. map the documents in the RDDs to their constituent ngrams (here we use the `mapPartitions` method with the `ngram_generator` defined above)
2. do the distributed key count using the `map` --> `(key, 1)` --> `reduceByKey` pattern
3. sort the result (in descending order) and take the top 1000 ngrams

In [ ]:
ngram_range = [1,3] # should use 1-5 ngram range, but make it smaller to speed up the processing a bit

# TODO: follow the three steps above -- remember that you need to pass the ngram_range to ngram_generator 
en_ngram_counts = (english_rdd.values()
                               <FILL IN>.cache())

In [ ]:
# TODO: follow the three steps above -- remember that you need to pass the ngram_range to ngram_generator
de_ngram_counts = (german_rdd.values()
                               <FILL IN>.cache())

In [ ]:
%%time
top_1000_en_ngrams = (en_ngram_counts.sortBy(lambda (ngram,count): count, False)
                                     .map(lambda (ngram, count): ngram)
                                     .take(1000))

In [ ]:
%%time
top_1000_de_ngrams = (de_ngram_counts.sortBy(lambda (ngram,count): count, False)
                                     .map(lambda (ngram, count): ngram)
                                     .take(1000))

In [ ]:
# building the top_ngrams dictionaries

# combine the german and english ngrams
top_ngrams = set(top_1000_de_ngrams) | set(top_1000_en_ngrams)

# build the ngrams dictionary lookup
top_ngrams_dict = {ngram:i for (i,ngram) in enumerate(top_ngrams)}

# broadcast the ngrams dictionary
top_ngrams_dict_b = sc.broadcast(top_ngrams_dict)

### Vectorizing the documents based on top 1000 ngrams from each language

For our logistic regression application, we are going to want to balance out the number of documents from the two languages. We know that English is by far over-represented, so we'll create a list of english and german document ID's that will balance out the two samples. 

In [ ]:
german_ids = cleaned_rdd.map(lambda (gid, _): gid).filter(lambda gid: meta_b.value[gid]['lang'] == 'de').collect()
english_ids = cleaned_rdd.map(lambda (gid, _): gid).filter(lambda gid: meta_b.value[gid]['lang'] == 'en').collect()

# here we ensure that the number of english ids is only as long as the list of german ids
combined_ids = set(german_ids + english_ids[:len(german_ids)]) 

In [ ]:
# create an RDD of german and english by only keeping the gids that are in combined_ids set
en_de_rdd = equalized_rdd.<FILL IN>

Now we can vectorize the documents just like we did in the previous notebook by using `vectorize_doc` with the broadcast vocabulary lookup we made above (`top_ngrams_dict_b` - remember to pass in its `value`!) and the pre-defined `ngram_range`. Be sure to change all spaces to '_' before passing in to `vectorize_doc` and to specify `ngram_type`. 

In [ ]:
# use vectorize_doc to map the en_de_rdd into (gid, vector) tuples 
# (remember to replace all spaces with '_' before passing text to vectorize_doc)
vector_rdd = (en_de_rdd.map(<FILL IN>
                                        .cache())

In [ ]:
%time vector_rdd.count()

Since we split the documents to equalize the vectorization load, we must now add the vectors for a given document back together: 

In [ ]:
# TODO: make 'combined_rdd' by calling 'reduceByKey' on 'vector_rdd' 
# to add up all of the vectors for a given gid
combined_rdd = vector_rdd.<FILL IN>

## Train the language classification model

Most Spark MLlib functions expect either plain vectors (for unsupervised learning) or `LabeledPoint` instances for supervised algorithms. Therefore, to train the model, we need to first map the `vector_rdd` elements into [`LabeledPoint`](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html?highlight=labeledpoint#pyspark.mllib.regression.LabeledPoint), which is just a Spark abstraction that encompases a *label* and a *vector*. We then split the data into a training and validation sets and produce the trained model. 

In [ ]:
from pyspark.mllib.feature import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD

In [ ]:
# making a simple LabeledPoint with label '0' and vector [1,2,3,4]
LabeledPoint(0, [1,2,3,4])

First, create a `vector_lp` RDD by mapping the contents of `vector_rdd` into a `LabeledPoint` using 0 if the language is English and 1 if it is German with the provided `gid_to_label` function:

In [ ]:
# TODO: create an RDD of LabeledPoint with 0 for english and 1 for german
def gid_to_label(gid):
    """Simple helper function that converts gid to 0 for english or 1 for german"""
    lang = meta_b.value[gid]['lang']
    if lang == 'en':
        return 0
    elif lang == 'de':
        return 1
    else:
        raise RuntimeError("Can't handle language '{lang}'".format(lang=lang))
    
vector_lp = combined_rdd.map(lambda (gid, vec): <FILL IN>)

The Spark machine learning library provides a simple method for creating training and validation sets, which we will use below. These will be our inputs for the logistic regression model fitting -- it is *always* a good idea to cache the inputs, since the training requires many iterations over the data. 

In [ ]:
training, validation = vector_lp.randomSplit([0.7,0.3])
training.cache().count()
validation.cache().count()

#### Pass the training set to the model

Here we will use the basic logistic regression model with stochastic gradient descent -- feel free to experiment with different parameters and other models from [pyspark.mllib.classification](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#module-pyspark.mllib.classification).

In [ ]:
model = LogisticRegressionWithSGD.train(training, regType='l1', regParam=0.2)

In [ ]:
plt.hist(model.weights, bins=100, range=[-50,50]);

To check the performance of the model, we define a function that takes a data RDD and a model as parameters and computes the error:

In [ ]:
def check_model(data, model) : 
    """Calculates the model error on the data
    
    Arguments: 
        
        data: the data RDD
        
        model: the classification model
        
    Returns:
    
        the error, which is the fraction of incorrectly predicted elements
    """
    
    error = (data.map(lambda p: (p.label, model.predict(p.features)))
                 .filter(lambda (v,p): v!=p).count())/float(data.count())
    return error

In [ ]:
train_error = check_model(training, model)
print("Training Error = " + str(train_error))
validation_error = check_model(validation, model)
print("Validation Error = " + str(validation_error))

For fun, lets create a function that will score a new string of text: 

In [ ]:
def predict_language(text, model) : 
    """Predict the language given the pre-trained model
    
    Arguments: 
        text: a string
        
        model: the trained logistic regression model
    """
    vec = vectorize_doc(text.replace(' ','_'), top_ngrams_dict_b.value, [1,2])
    
    res = 'german' if model.predict(vec) == 1 else 'english'
    print text + ' is ' + res

predict_lang_partial = lambda text: predict_language(text, model)

Try it out! Enter your own sentence:

In [ ]:
interact(predict_lang_partial, text='a dog bit me')

### Which character sequences define each language? 

Here we'll use the weights of the trained model to identify the words with the lowest weights (i.e. those that pull a prediction toward zero) and the highest weights (those that pull a prediction toward 1). 

In [ ]:
sorted_weights = np.argsort(model.weights)

top_ngram_inds = {v:k for k,v in top_ngrams_dict.iteritems()}

print('top 50 english and german ngrams:')
for i in range(50): 
    eng_ind = sorted_weights[i]
    ger_ind = sorted_weights[len(top_ngram_inds)-i-1]
    print('%s\t%d\t%s\t%d'%(top_ngram_inds[eng_ind], model.weights[eng_ind], 
                            top_ngram_inds[ger_ind], model.weights[ger_ind]))

Some other ideas for things you could do with this dataset: 

* try other [classifiers that are included in MLlib](http://spark.apache.org/docs/latest/mllib-classification-regression.html)
* build a regression model to predict year of publication (may be better with word ngrams)
* run a clustering algorithm on the full language data to identify all the languages present
* do clustering on the english books and see if sub-groups of the language pop up
* cluster by author -- do certain authors write in similar ways?

In [ ]:
sc.stop()